In [16]:
import os
import glob
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM, Embedding, Dropout
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import EarlyStopping ,ModelCheckpoint
from sklearn.preprocessing import StandardScaler


print('lstm')

lstm


In [8]:
day=1
print('day', day)

files = glob.glob("../stock_dataset_with_label/2330/ask/*")
# print(files)

X_arr, y_arr = [], []

for f in files:
    data = pd.read_csv(f, index_col=None)
    data = data.drop(columns=['matchPri','openPri','bidPri1','bidPri2','bidPri3','bidPri4','bidPri5','askPri1','askPri2','askPri3','askPri4','askPri5'])
    X_data = data.drop(columns=['label'])
    X_data = X_data.to_numpy()
    y_data = data['label'].to_numpy()

    pastDay = day
    futureDay = 1

    for i in range(pastDay, data.shape[0]-futureDay):
        X_temp = X_data[i-pastDay : i]
        X_arr.append(X_temp)
        y_arr.append(y_data[i])
        


day 1


In [10]:
X_arr = np.array(X_arr)
y_arr = np.array(y_arr)

length = len(X_arr)
print(len(X_arr))
print(len(y_arr))

train_length = int(length*7/10)
X_train = X_arr[:train_length]
y_train = y_arr[:train_length]
X_test = X_arr[train_length:]
y_test = y_arr[train_length:]


# balanced weight 處理 平盤過多的問題 
class_weights = compute_class_weight('balanced', classes = np.unique(y_train) ,y= y_train)
class_weights_dict = {i: class_weights[i] for i in range(len(class_weights))}
print(class_weights_dict)


y_train = keras.utils.to_categorical(y_train, num_classes=2)
y_test = keras.utils.to_categorical(y_test, num_classes=2)

print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)










527057
527057
{0: 0.5998598469698004, 1: 3.003508743365137}
(368939, 1, 11) (158118, 1, 11)
(368939, 2) (158118, 2)


In [14]:
model = Sequential()

model.add(LSTM(units = 32, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units = 32, return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(units = 32))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))
model.compile(optimizer=SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['acc'])
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 1, 32)             5632      
                                                                 
 dropout_12 (Dropout)        (None, 1, 32)             0         
                                                                 
 lstm_13 (LSTM)              (None, 1, 32)             8320      
                                                                 
 dropout_13 (Dropout)        (None, 1, 32)             0         
                                                                 
 lstm_14 (LSTM)              (None, 32)                8320      
                                                                 
 dropout_14 (Dropout)        (None, 32)                0         
                                                                 
 dense_4 (Dense)             (None, 2)                

In [15]:
checkpoint =ModelCheckpoint(
    filepath="../LSTM_best.h5",
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_best_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=10 ,min_delta=0.0001)
model.fit(X_train, y_train, epochs = 100, batch_size = 32, class_weight = class_weights_dict,validation_split=0.2,callbacks=[checkpoint, early_stop])
# model.fit(X_train, y_train, epochs = 100, batch_size = 32,validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',patience=7, min_delta=0.0001)])


Epoch 1/100
9224/9224 [==============================] - 98s 10ms/step - loss: 0.6913 - acc: 0.5925 - val_loss: 0.6858 - val_acc: 0.8292
Epoch 2/100
9224/9224 [==============================] - 95s 10ms/step - loss: 0.6913 - acc: 0.6255 - val_loss: 0.6934 - val_acc: 0.4702
Epoch 3/100
9224/9224 [==============================] - 96s 10ms/step - loss: 0.6912 - acc: 0.6304 - val_loss: 0.6942 - val_acc: 0.3959
Epoch 4/100
9224/9224 [==============================] - 94s 10ms/step - loss: 0.6912 - acc: 0.5996 - val_loss: 0.6905 - val_acc: 0.6356
Epoch 5/100
1559/9224 [====>.........................] - ETA: 1:20 - loss: 0.6913 - acc: 0.6827

KeyboardInterrupt: 

In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score 
pred=model.predict(X_test)
pred = (pred > 0.5) 


5392/5392 [==============================] - 15s 3ms/step


ValueError: Target is multilabel-indicator but average='binary'. Please choose another average setting, one of [None, 'micro', 'macro', 'weighted', 'samples'].

In [11]:
new_pred = []
for i in pred:
    if i[0]:
        new_pred.append(0)
    else:
        new_pred.append(1)

print(new_pred)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [7]:
print(pred)

[[ True False]
 [ True False]
 [ True False]
 ...
 [ True False]
 [ True False]
 [False  True]]


In [12]:
print(f"Precision Score: {precision_score(y_test, new_pred)*100:.2f}%")
print(f"Recall Score: {recall_score(y_test, new_pred)*100:.2f}%")
print(f"F1 score: {f1_score(y_test, new_pred)*100:.2f}%")

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets

In [ ]:
trainPredict = model.predict(X_train)
testPredict = model.predict(X_test)

print(trainPredict)
print(testPredict)